In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from catboost import CatBoostClassifier
import glob, os
import matplotlib.pyplot as plt
import warnings
from pandas_profiling import ProfileReport
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 150)
pd.set_option('display.max_colwidth', 150)

import seaborn as sns
if os.name == 'nt' : # Windows OS
    font_family= "Malgun Gothic"
else: # MAC OS
    font_family= "AppleGothic"
# - 값이 깨지는 문제 해결을 위해 파라미터값 설정
sns.set(font = font_family, rc = {"axes.unicode_minus" : False})

### 1. 데이터 로드 및 확인

In [ ]:
#x = pd.read_csv("track_b_learn.csv").drop(["idx"],axis=1)
#x2 = pd.read_csv("track_b_pre.csv").drop(['idx'], axis=1)

x = pd.read_csv("track_b_learn.csv").drop(["idx"],axis=1)
x2 = pd.read_csv("track_b_pre.csv").drop(['idx'], axis=1)
y = pd.read_csv("track_b_learn_label.csv").drop(columns = ["idx"],axis =1)
display(x.head(), y.head())

s_ip  s_port           d_ip  d_port  protocol  \
0  210.248.110.200   47760  144.82.144.26   42718       TCP   
1   172.116.62.150   47760     38.29.1.69   39548       TCP   
2    15.164.135.69   60390  37.154.75.207      80       TCP   
3    84.12.150.116   47760   64.89.252.59   37816       TCP   
4    95.42.196.145   53040  17.75.195.163      80  TCP( 6 )   

                                                                                                                                                 payload  \
0                                                                                                                                                ......(   
1                                                                                                                                                ......(   
2  GET /?id=-9865%27%29%20UNION%20ALL%20SELECT%207212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C72...   
3                                                                                                                                                ......(   
4  GET /cgi_wrapper HTTP/1.1\r\nReferer: () { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }\r\nHost: 172.16.1.50\r\nConnection: Kee...   

   risk  
0     2  
1     2  
2     1  
3     2  
4     1

class
0  4_unknown
1  4_unknown
2  2_exploit
3  4_unknown
4  2_exploit

In [ ]:
x.reset_index(inplace = True, drop=True)
x2.reset_index(inplace = True, drop=True)

In [ ]:
x.head()

s_ip  s_port           d_ip  d_port  protocol  \
0  210.248.110.200   47760  144.82.144.26   42718       TCP   
1   172.116.62.150   47760     38.29.1.69   39548       TCP   
2    15.164.135.69   60390  37.154.75.207      80       TCP   
3    84.12.150.116   47760   64.89.252.59   37816       TCP   
4    95.42.196.145   53040  17.75.195.163      80  TCP( 6 )   

                                                                                                                                                 payload  \
0                                                                                                                                                ......(   
1                                                                                                                                                ......(   
2  GET /?id=-9865%27%29%20UNION%20ALL%20SELECT%207212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C72...   
3                                                                                                                                                ......(   
4  GET /cgi_wrapper HTTP/1.1\r\nReferer: () { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }\r\nHost: 172.16.1.50\r\nConnection: Kee...   

   risk  
0     2  
1     2  
2     1  
3     2  
4     1

In [ ]:
x2.head()

s_ip  s_port           d_ip  d_port  protocol  \
0  194.45.133.205    3389  223.78.72.205   39034  TCP( 6 )   
1   41.122.42.110    3389  56.139.100.11   43746  TCP( 6 )   
2   114.223.201.8   43330  97.209.139.46      80  TCP( 6 )   
3  174.134.63.141      80  155.18.68.107    9155  TCP( 6 )   
4   9.100.170.255   59254  169.9.189.151      80       TCP   

                                                                                                                                                 payload  \
0                                                                                                                                    .........4.........   
1  .........4.............A...M..a..-3.].n....b..[z.^w...!...=... ......m..Td...g..r....C`q.....n../..................0...0..........a....i_.O...Ae.....   
2  OPTIONS / HTTP/1.1\r\nHost: 172.16.1.50\r\nUser-Agent: Mozilla/5.0 (compatible; Nmap Scripting Engine; https://nmap.org/book/nse.html)\r\nConnecti...   
3  HTTP/1.1 200 OK\r\nCache-Control: private\r\nContent-Type: text/html\r\nServer: Microsoft-IIS/7.5\r\nX-Powered-By: ASP.NET\r\nDate: Fri, 23 Jul 20...   
4  GET /?id=-7327%27%29%20AS%20pCEU%20WHERE%209573%3D9573%20UNION%20ALL%20SELECT%209573%2C9573%2C9573%2C9573%2C9573%2C9573%2C9573%2C9573%2C9573%2C957...   

   risk  
0     3  
1     3  
2     1  
3     1  
4     1

In [ ]:
x.tail()

s_ip  s_port            d_ip  d_port protocol  payload  risk
49995   128.194.190.9   47760   90.155.31.230   44958      TCP  ......(     2
49996    58.186.58.89   47760  202.147.92.191   33206      TCP  ......(     2
49997    75.241.39.21   47760  184.243.14.105   55558      TCP  ......(     2
49998  146.204.158.13   47760    96.204.22.41   39450      TCP  ......(     2
49999   113.192.4.211   47760   95.42.130.186   36302      TCP  ......(     2

In [ ]:
x = pd.concat([x, x2], axis=0)
x.reset_index(inplace = True, drop=True)

In [ ]:
x.tail()

s_ip  s_port             d_ip  d_port  protocol  \
59995     152.44.75.69   20688     116.45.96.46      80  TCP( 6 )   
59996   72.218.153.161   48932    65.236.97.204      80       TCP   
59997    40.176.190.42   47760    35.145.95.220   34038       TCP   
59998  128.141.191.152   49962  139.104.242.217    5985  TCP( 6 )   
59999   193.50.137.161   52886      63.4.101.58      80  TCP( 6 )   

                                                                                                                                                     payload  \
59995  GET /index.JSP HTTP/1.1\r\nConnection: Keep-Alive\r\nHost: 172.16.1.50\r\nContent-Length: 714\r\nUser-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:...   
59996  GET /?id=user%22%20UNION%20ALL%20SELECT%20NULL%2CNULL%2CNULL%2CNULL%2CNULL%2CNULL%2CNULL%2CNULL%2CNULL--%20yzCW HTTP/1.1\r\nAccept-Encoding: gzip,...   
59997                                                                                                                                                ......(   
59998  t/encrypted;protocol=\"application/HTTP-SPNEGO-session-encrypted\";boundary=\"Encrypted Boundary\"\r\nUser-Agent: Microsoft WinRM Client\r\nConten...   
59999  GET /cgi-sys/tmUnblock.cgi HTTP/1.1\r\nUser-Agent: () { :; }; echo 93e4r0-CVE-2014-6271: true;echo;echo;\r\nHost: 172.16.1.50\r\nConnection: Keep-...   

       risk  
59995     1  
59996     1  
59997     2  
59998     2  
59999     1

In [ ]:
df = pd.concat([x,y],axis =1)

In [ ]:
dftest = df.copy()

In [ ]:
profile = ProfileReport(dftest)
profile

### 2. 파생변수 생성 

In [ ]:
# ip 주소 쪼개서 파생변수 생성

x["s_ip1"] = x["s_ip"].str.split(".").str.get(0)
x["s_ip2"] = x["s_ip"].str.split(".").str.get(1)
x["s_ip3"] = x["s_ip"].str.split(".").str.get(2)
x["s_ip4"] = x["s_ip"].str.split(".").str.get(3)

x["d_ip1"] = x["d_ip"].str.split(".").str.get(0)
x["d_ip2"] = x["d_ip"].str.split(".").str.get(1)
x["d_ip3"] = x["d_ip"].str.split(".").str.get(2)
x["d_ip4"] = x["d_ip"].str.split(".").str.get(3)

In [ ]:
# ip 주소 host id, network id 분류 파생변수 생성

x["s_host_ip"]= x["s_ip2"].astype("str") + " " + x["s_ip3"].astype("str") + " " + x["s_ip4"].astype("str")
x["d_host_ip"]= x["d_ip2"].astype("str") + " " + x["d_ip3"].astype("str") + " " + x["d_ip4"].astype("str")
x = x.rename(columns = {"s_ip1" : "s_network_id", "d_ip1" : "d_network_id"})
x.head()

s_ip  s_port           d_ip  d_port  protocol  \
0  210.248.110.200   47760  144.82.144.26   42718       TCP   
1   172.116.62.150   47760     38.29.1.69   39548       TCP   
2    15.164.135.69   60390  37.154.75.207      80       TCP   
3    84.12.150.116   47760   64.89.252.59   37816       TCP   
4    95.42.196.145   53040  17.75.195.163      80  TCP( 6 )   

                                                                                                                                                 payload  \
0                                                                                                                                                ......(   
1                                                                                                                                                ......(   
2  GET /?id=-9865%27%29%20UNION%20ALL%20SELECT%207212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C72...   
3                                                                                                                                                ......(   
4  GET /cgi_wrapper HTTP/1.1\r\nReferer: () { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }\r\nHost: 172.16.1.50\r\nConnection: Kee...   

   risk s_network_id s_ip2 s_ip3 s_ip4 d_network_id d_ip2 d_ip3 d_ip4  \
0     2          210   248   110   200          144    82   144    26   
1     2          172   116    62   150           38    29     1    69   
2     1           15   164   135    69           37   154    75   207   
3     2           84    12   150   116           64    89   252    59   
4     1           95    42   196   145           17    75   195   163   

     s_host_ip   d_host_ip  
0  248 110 200   82 144 26  
1   116 62 150     29 1 69  
2   164 135 69  154 75 207  
3   12 150 116   89 252 59  
4   42 196 145  75 195 163

In [ ]:
# IP 헤더에 따른 클래스 파생변수 생성

x.iloc[:,7:15] = x.iloc[:,7:15].astype("int") # 변수 타입 정수로 설정

x["s_ip_class"] = np.nan
x["d_ip_class"] = np.nan

x["s_ip_class"] = np.where(x["s_network_id"] <= 127 , "A", x["s_ip_class"])
x["s_ip_class"] = np.where((x["s_network_id"] >= 128) & (x["s_network_id"] <= 191) , "B", x["s_ip_class"])
x["s_ip_class"] = np.where((x["s_network_id"] >= 192) & (x["s_network_id"] <= 223) , "C", x["s_ip_class"])
x["s_ip_class"] = np.where((x["s_network_id"] >= 224) , "D or E", x["s_ip_class"])

x["d_ip_class"] = np.where(x["d_network_id"] <= 127 , "A", x["d_ip_class"])
x["d_ip_class"] = np.where((x["d_network_id"] >= 128) & (x["d_network_id"] <= 191) , "B", x["d_ip_class"])
x["d_ip_class"] = np.where((x["d_network_id"] >= 192) & (x["d_network_id"] <= 223) , "C", x["d_ip_class"])
x["d_ip_class"] = np.where((x["d_network_id"] >= 224) , "D or E", x["d_ip_class"])

In [ ]:
# 포트번호에 따른 클래스 파생변수 생성

x["s_port_class"] = np.nan
x["d_port_class"] = np.nan

x["s_port_class"] = np.where(x["s_port"] <= 1023 , "well-known", x["s_port_class"])
x["s_port_class"] = np.where((x["s_port"] >= 1024) & (x["s_port"] <= 49151) , "registered", x["s_port_class"])
x["s_port_class"] = np.where((x["s_port"] >= 49152) & (x["s_port"] <= 65535) , "dynamic", x["s_port_class"])

x["d_port_class"] = np.where(x["d_port"] <= 1023 , "well-known", x["d_port_class"])
x["d_port_class"] = np.where((x["d_port"] >= 1024) & (x["d_port"] <= 49151) , "registered", x["d_port_class"])
x["d_port_class"] = np.where((x["d_port"] >= 49152) & (x["d_port"] <= 65535) , "dynamic", x["d_port_class"])

#### TfidfVectorizer의 tokenizer인자로 사용될 lemmatization 어근 변환 함수를 설정. 

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 입력으로 들어온 token단어들에 대해서 lemmatization 어근 변환. 
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# TfidfVectorizer 객체 생성 시 tokenizer인자로 해당 함수를 설정하여 lemmatization 적용
# 입력으로 문장을 받아서 stop words 제거-> 소문자 변환 -> 단어 토큰화 -> lemmatization 어근 변환. 
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
nltk.download("punkt")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASNKMGSK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASNKMGSK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASNKMGSK\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

#### TF-IDF 기반 Vectorization 적용 및 KMeans 군집화 수행. 
* Stemming과 Lemmatization 같은 어근 변환은 TfidfVectorizer에서 직접 지원하진 않으나 tokenizer 파라미터에 커스텀 어근 변환 함수를 적용하여 어근 변환을 수행할 수 있음
* TfidfVectorizer 생성자의 tokenizer인자로 위에서 생성 LemNormalize 함수 설정.

In [ ]:
x.fillna(0,inplace = True)
x['payload'] = x['payload'].astype(str)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

tfidf = tfidf_vect.fit(x["payload"])
tfidf.get_feature_names()
#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.transform(x['payload'])

In [ ]:
# tfidf 행렬 -> 데이터프레임 변환

pd.DataFrame(feature_vect.toarray(), columns = tfidf.get_feature_names())

0  0 color  0px  0px 2px  0px 4px  0px 5px  0px 6px  0px abc  \
0      0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
...    ...      ...  ...      ...      ...      ...      ...      ...   
59995  0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
59996  0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
59997  0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
59998  0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   
59999  0.0      0.0  0.0      0.0      0.0      0.0      0.0      0.0   

       0rgba1741741831  0rgba1741741831 100    1  1 fontsize  100  100 color  \
0                  0.0                  0.0  0.0         0.0  0.0        0.0   
1                  0.0                  0.0  0.0         0.0  0.0        0.0   
2                  0.0                  0.0  0.0         0.0  0.0        0.0   
3                  0.0                  0.0  0.0         0.0  0.0        0.0   
4                  0.0                  0.0  0.0         0.0  0.0        0.0   
...                ...                  ...  ...         ...  ...        ...   
59995              0.0                  0.0  0.0         0.0  0.0        0.0   
59996              0.0                  0.0  0.0         0.0  0.0        0.0   
59997              0.0                  0.0  0.0         0.0  0.0        0.0   
59998              0.0                  0.0  0.0         0.0  0.0        0.0   
59999              0.0                  0.0  0.0         0.0  0.0        0.0   

       100 main  100 w3c  10px  10px rgba000015  11en  \
0           0.0      0.0   0.0              0.0   0.0   
1           0.0      0.0   0.0              0.0   0.0   
2           0.0      0.0   0.0              0.0   0.0   
3           0.0      0.0   0.0              0.0   0.0   
4           0.0      0.0   0.0              0.0   0.0   
...         ...      ...   ...              ...   ...   
59995       0.0      0.0   0.0              0.0   0.0   
59996       0.0      0.0   0.0              0.0   0.0   
59997       0.0      0.0   0.0              0.0   0.0   
59998       0.0      0.0   0.0              0.0   0.0   
59999       0.0      0.0   0.0              0.0   0.0   

       11en httpwwww3orgtrxhtml11dtdxhtml11dtdhtmlhead  123h1  123h1 p  128px  \
0                                                  0.0    0.0      0.0    0.0   
1                                                  0.0    0.0      0.0    0.0   
2                                                  0.0    0.0      0.0    0.0   
3                                                  0.0    0.0      0.0    0.0   
4                                                  0.0    0.0      0.0    0.0   
...                                                ...    ...      ...    ...   
59995                                              0.0    0.0      0.0    0.0   
59996                                              0.0    0.0      0.0    0.0   
59997                                              0.0    0.0      0.0    0.0   
59998                                              0.0    0.0      0.0    0.0   
59999                                              0.0    0.0      0.0    0.0   

       128px fontweight  132058  132058 gmt  13215058a1e728280  \
0                   0.0     0.0         0.0                0.0   
1                   0.0     0.0         0.0                0.0   
2                   0.0     0.0         0.0                0.0   
3                   0.0     0.0         0.0                0.0   
4                   0.0     0.0         0.0                0.0   
...                 ...     ...         ...                ...   
59995               0.0     0.0         0.0        

In [ ]:
'''
from sklearn.cluster import DBSCAN

# epsilon, 최소 샘플 개수 설정
dbs = DBSCAN(eps=0.3, min_samples=10)

# 군집화 모델 학습 및 클러스터 예측 결과 반환
x['is_outlier'] = dbs.fit_predict(feature_vect)
x["is_outlier"] = np.where(x["is_outlier"] != -1, 0 , x["is_outlier"])
'''

'\nfrom sklearn.cluster import DBSCAN\n\n# epsilon, 최소 샘플 개수 설정\ndbs = DBSCAN(eps=0.3, min_samples=10)\n\n# 군집화 모델 학습 및 클러스터 예측 결과 반환\nx[\'is_outlier\'] = dbs.fit_predict(feature_vect)\nx["is_outlier"] = np.where(x["is_outlier"] != -1, 0 , x["is_outlier"])\n'

In [ ]:
from sklearn.cluster import KMeans

# 43개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=999
km_cluster = KMeans(n_clusters = 43, max_iter=10000, random_state=999)
km_cluster.fit(feature_vect)

KMeans(max_iter=10000, n_clusters=43, random_state=999)

In [ ]:
# 클러스터링 모델 저장
import joblib
#joblib.dump(km_cluster, "km_cluster.pkl")

In [ ]:
'''
km_cluster = joblib.load("km_cluster.pkl")
cluster_label = km_cluster.predict(feature_vect)
cluster_label
'''

'\nkm_cluster = joblib.load("km_cluster.pkl")\ncluster_label = km_cluster.predict(feature_vect)\ncluster_label\n'

In [ ]:
x['cluster_label_kmeans'] = km_cluster.predict(feature_vect)
x.head()

s_ip  s_port           d_ip  d_port  protocol  \
0  210.248.110.200   47760  144.82.144.26   42718       TCP   
1   172.116.62.150   47760     38.29.1.69   39548       TCP   
2    15.164.135.69   60390  37.154.75.207      80       TCP   
3    84.12.150.116   47760   64.89.252.59   37816       TCP   
4    95.42.196.145   53040  17.75.195.163      80  TCP( 6 )   

                                                                                                                                                 payload  \
0                                                                                                                                                ......(   
1                                                                                                                                                ......(   
2  GET /?id=-9865%27%29%20UNION%20ALL%20SELECT%207212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C72...   
3                                                                                                                                                ......(   
4  GET /cgi_wrapper HTTP/1.1\r\nReferer: () { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }\r\nHost: 172.16.1.50\r\nConnection: Kee...   

   risk  s_network_id  s_ip2  s_ip3  s_ip4  d_network_id  d_ip2  d_ip3  d_ip4  \
0     2           210    248    110    200           144     82    144     26   
1     2           172    116     62    150            38     29      1     69   
2     1            15    164    135     69            37    154     75    207   
3     2            84     12    150    116            64     89    252     59   
4     1            95     42    196    145            17     75    195    163   

     s_host_ip   d_host_ip s_ip_class d_ip_class s_port_class d_port_class  \
0  248 110 200   82 144 26          C          B   registered   registered   
1   116 62 150     29 1 69          B          A   registered   registered   
2   164 135 69  154 75 207          A          A      dynamic   well-known   
3   12 150 116   89 252 59          A          A   registered   registered   
4   42 196 145  75 195 163          A          A      dynamic   well-known   

   cluster_label_kmeans  
0                     1  
1                     1  
2                     2  
3                     1  
4                    18

In [ ]:
x.shape

(60000, 22)

In [ ]:
x.to_csv('tfid.csv')

#### 공격여부 변수 생성

In [ ]:
import re
import urllib.parse
import pprint

In [ ]:
idxs = x.index
payloads = x["payload"]
label = y["class"]

In [ ]:
# Reconnaissance
def is_xml_rpc(s: str):
    # https://nitesculucian.github.io/2019/07/01/exploiting-the-xmlrpc-php-on-all-wordpress-versions/
    return "<methodCall>" in s

def is_shell_shock(s: str):
    # https://github.com/sullo/nikto/blob/master/program/plugins/nikto_shellshock.plugin
    # https://kinggod-daddong.tistory.com/4
    return "() { :; }; echo" in s

# Execution
def is_sql_injection(s: str):  # Initial Access -> Exploit Public-Facing Application
    dmls = ["insert", "update", "delete", "select", "INSERT", "UPDATE", "DELETE", "SELECT"]
    return int(any(list(map(lambda d: d in s, dmls))))

def is_malicious_execution(s: str):
    return "/etc/passwd" in s

def is_xss_attack(s: str):
    return "<script>" in s

def is_directory_traversal(s: str):
    # https://www.cvedetails.com/cve/CVE-2005-1198/
    return "apexec.pl" in s

# Post
def is_web_shell(s: str):  # Persistence
    # https://github.com/Yara-Rules/rules/blob/master/webshells/WShell_THOR_Webshells.yar
    return "aspydrv" in s

def is_shell_file(s: str):  # Persistence
    # https://lannstark.tistory.com/8
    # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Content-Disposition
    return "shell" in s

In [ ]:
methods = ["GET", "HEAD", "POST", "PUT", "DELETE", "CONNECT", "OPTIONS", "TRACE"]
d_keys = ["has_xml_rpc", "has_sql_injection", "has_shell_shock", "has_malicious_execution", "has_xss_attack", "has_directory_traversal", "has_web_shell", "has_shell_file"]
d_values = [0,0,0,0,0,0,0,0]

In [ ]:
dicts = dict()

for idx in tqdm(range(x.shape[0])):
    payload = payloads[idx]
    # print(f"{idx}\n{payload}\n")
    d = dict(zip(d_keys, d_values))
    
    if payload is np.nan:
        dicts.update({idx: d})
        continue
    elif "DOCTYPE" in payload:
        d.update({"has_html": 1})
        dicts.update({idx : d})
        continue

    fields = re.split("\\r\\n|\\\\r\\\\n", payload)
    for field in fields:
        if not field:
            if d:
                dicts.update({idx: d})
            else:
                d = dict(zip(d_keys, d_values))
        #print(field)
        field_name, *field_value = field.split(": ")
        if is_xml_rpc(field_name):
            d.update({d_keys[0]: 1})
        elif "Content-Disposition" in field_name and any(map(lambda v: is_shell_file(v), field_value)):
            d.update({d_keys[7]: 1})
        elif "User-Agent" in field_name and any(map(lambda v: is_shell_shock(v), field_value)):
            d.update({d_keys[2]: 1})
        elif any(m in field_name for m in methods): # for start line of the HTTP message
            # if is_xss_attack(field_name):
            #     d.update({d_keys[4]: 1})
            #     continue
            if is_directory_traversal(field_name):
                d.update({d_keys[5]: 1})
                continue

            parse_result = urllib.parse.urlparse(field_name)
            query = parse_result.query
            queries = urllib.parse.parse_qs(query)
            for param_name, param_value in queries.items():
                if any(map(lambda v: is_sql_injection(v), param_value)):
                    d.update({d_keys[1]: 1}) 
                elif any(map(lambda v: is_malicious_execution(v), param_value)):
                    d.update({d_keys[3]: 1})
                elif is_xss_attack(field_name):
                    d.update({d_keys[4]: 1})
        # elif any(map(lambda v: is_shell_shock(v), field_value)):
        #     d.update({d_keys[2]: 1})
        elif any(map(lambda v: is_web_shell(v), field_value)):
            d.update({d_keys[6]: 1})
        
    value = dicts.get(idx)
    if value is None:
        dicts.update({idx: d})

#print()
#pprint.pprint(dicts)

100%|██████████████████████████████████████████████████████████████████████████| 60000/60000 [00:07<00:00, 8496.12it/s]


In [ ]:
dl = {'has_sql_injection' : [],
      'has_shell_shock' : [],
      'has_malicious_execution' : [],
      'has_xss_attack' : [],
      'has_web_shell' : [],
      'has_xml_rpc': [],
      'has_directory_traversal': [],
      'has_shell_file': []
      }
dl

{'has_sql_injection': [],
 'has_shell_shock': [],
 'has_malicious_execution': [],
 'has_xss_attack': [],
 'has_web_shell': [],
 'has_xml_rpc': [],
 'has_directory_traversal': [],
 'has_shell_file': []}

In [ ]:
len(dicts)

60000

In [ ]:
# 여러 딕셔너리로 저장된 값을 하나의 딕셔너리로 저장

for i in tqdm(range(len(dicts))):
    dl["has_sql_injection"].append(dicts[i]["has_sql_injection"])
    dl["has_shell_shock"].append(dicts[i]["has_shell_shock"])
    dl["has_malicious_execution"].append(dicts[i]["has_malicious_execution"])
    dl["has_xss_attack"].append(dicts[i]["has_xss_attack"])
    dl["has_web_shell"].append(dicts[i]["has_web_shell"])
    dl["has_xml_rpc"].append(dicts[i]["has_xml_rpc"])
    dl["has_directory_traversal"].append(dicts[i]["has_directory_traversal"])
    dl["has_shell_file"].append(dicts[i]["has_shell_file"])


100%|████████████████████████████████████████████████████████████████████████| 60000/60000 [00:00<00:00, 498442.71it/s]


In [ ]:
# 페이로드 기반 공격 여부 변수 생성

x["has_sql_injection"] = dl["has_sql_injection"]
x["has_shell_shock"] = dl["has_shell_shock"]
x["has_malicious_execution"] = dl["has_malicious_execution"]
x["has_xss_attack"] = dl["has_xss_attack"]
x["has_web_shell"] = dl["has_web_shell"]
x["has_xml_rpc"] = dl["has_xml_rpc"]
x["has_directory_traversal"] = dl["has_directory_traversal"]
x["has_shell_file"] = dl["has_shell_file"]

In [ ]:
# 설명변수 모두 범주형 변수로 변환(수치형으로 보이는 자료 없음)
x = x.astype(object)
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   s_ip                     60000 non-null  object
 1   s_port                   60000 non-null  object
 2   d_ip                     60000 non-null  object
 3   d_port                   60000 non-null  object
 4   protocol                 60000 non-null  object
 5   payload                  60000 non-null  object
 6   risk                     60000 non-null  object
 7   s_network_id             60000 non-null  object
 8   s_ip2                    60000 non-null  object
 9   s_ip3                    60000 non-null  object
 10  s_ip4                    60000 non-null  object
 11  d_network_id             60000 non-null  object
 12  d_ip2                    60000 non-null  object
 13  d_ip3                    60000 non-null  object
 14  d_ip4                    60000 non-nul

In [ ]:
df = pd.read_csv("ip_location.csv")
df.head()

idx s_country_code d_country_code
0    0             JP             GB
1    1             US             US
2    2             KR             TR
3    3             GB             US
4    4             BG             ES

In [ ]:
x = pd.concat([x, df.iloc[:,1:]],axis=1)
x.head()

s_ip s_port           d_ip d_port  protocol  \
0  210.248.110.200  47760  144.82.144.26  42718       TCP   
1   172.116.62.150  47760     38.29.1.69  39548       TCP   
2    15.164.135.69  60390  37.154.75.207     80       TCP   
3    84.12.150.116  47760   64.89.252.59  37816       TCP   
4    95.42.196.145  53040  17.75.195.163     80  TCP( 6 )   

                                                                                                                                                 payload  \
0                                                                                                                                                ......(   
1                                                                                                                                                ......(   
2  GET /?id=-9865%27%29%20UNION%20ALL%20SELECT%207212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C7212%2C72...   
3                                                                                                                                                ......(   
4  GET /cgi_wrapper HTTP/1.1\r\nReferer: () { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }\r\nHost: 172.16.1.50\r\nConnection: Kee...   

  risk s_network_id s_ip2 s_ip3 s_ip4 d_network_id d_ip2 d_ip3 d_ip4  \
0    2          210   248   110   200          144    82   144    26   
1    2          172   116    62   150           38    29     1    69   
2    1           15   164   135    69           37   154    75   207   
3    2           84    12   150   116           64    89   252    59   
4    1           95    42   196   145           17    75   195   163   

     s_host_ip   d_host_ip s_ip_class d_ip_class s_port_class d_port_class  \
0  248 110 200   82 144 26          C          B   registered   registered   
1   116 62 150     29 1 69          B          A   registered   registered   
2   164 135 69  154 75 207          A          A      dynamic   well-known   
3   12 150 116   89 252 59          A          A   registered   registered   
4   42 196 145  75 195 163          A          A      dynamic   well-known   

  cluster_label_kmeans has_sql_injection has_shell_shock  \
0                    1                 0               0   
1                    1                 0               0   
2                    2                 1               0   
3                    1                 0               0   
4                   18                 0               1   

  has_malicious_execution has_xss_attack has_web_shell has_xml_rpc  \
0                       0              0             0           0   
1                       0              0             0           0   
2                       0              0             0           0   
3                       0              0             0           0   
4                       0              0             0           0   

  has_directory_traversal has_shell_file s_country_code d_country_code  
0                       0              0             JP             GB  
1                       0              0             US             US  
2                       0              0             KR             TR  
3                       0              0             GB             US  
4                       0              0             BG             ES

In [ ]:
x["s_country_code"] = x["s_country_code"].replace(np.nan, "unknown")
x["d_country_code"] = x["d_country_code"].replace(np.nan, "unknown")

In [ ]:
# 임시 저장
x.to_csv("train_touch.csv", index = False)

### 3. 만든 변수를 가지고 모델링(catboost)

In [ ]:
# drop : "s_ip" "d_host_ip" "has_web_shell"
# macro precision : 0.98688 macro f1-score: 0.98564
x = pd.read_csv("train_touch.csv").drop(columns = ["s_ip", "d_host_ip"],axis =1)
x["attack_pattern"] = x["has_sql_injection"] + x["has_shell_shock"] + x["has_malicious_execution"] + x["has_xss_attack"] + x["has_web_shell"] + x["has_xml_rpc"] + x["has_directory_traversal"] + x["has_shell_file"]
x = pd.read_csv("train_touch.csv").drop(columns = ["payload","has_web_shell", "s_ip", "d_ip", 's_country_code', 'd_country_code'], axis =1)
x = x.astype(object)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   s_port                   60000 non-null  object
 1   d_port                   60000 non-null  object
 2   protocol                 60000 non-null  object
 3   risk                     60000 non-null  object
 4   s_network_id             60000 non-null  object
 5   s_ip2                    60000 non-null  object
 6   s_ip3                    60000 non-null  object
 7   s_ip4                    60000 non-null  object
 8   d_network_id             60000 non-null  object
 9   d_ip2                    60000 non-null  object
 10  d_ip3                    60000 non-null  object
 11  d_ip4                    60000 non-null  object
 12  s_host_ip                60000 non-null  object
 13  d_host_ip                60000 non-null  object
 14  s_ip_class               60000 non-nul

In [ ]:
xcopy = x.copy()
x = xcopy.iloc[:50000, :]
x2 = xcopy.iloc[50000:, :]

In [ ]:
x

s_port d_port  protocol risk s_network_id s_ip2 s_ip3 s_ip4  \
0      47760  42718       TCP    2          210   248   110   200   
1      47760  39548       TCP    2          172   116    62   150   
2      60390     80       TCP    1           15   164   135    69   
3      47760  37816       TCP    2           84    12   150   116   
4      53040     80  TCP( 6 )    1           95    42   196   145   
...      ...    ...       ...  ...          ...   ...   ...   ...   
49995  47760  44958       TCP    2          128   194   190     9   
49996  47760  33206       TCP    2           58   186    58    89   
49997  47760  55558       TCP    2           75   241    39    21   
49998  47760  39450       TCP    2          146   204   158    13   
49999  47760  36302       TCP    2          113   192     4   211   

      d_network_id d_ip2 d_ip3 d_ip4    s_host_ip   d_host_ip s_ip_class  \
0              144    82   144    26  248 110 200   82 144 26          C   
1               38    29     1    69   116 62 150     29 1 69          B   
2               37   154    75   207   164 135 69  154 75 207          A   
3               64    89   252    59   12 150 116   89 252 59          A   
4               17    75   195   163   42 196 145  75 195 163          A   
...            ...   ...   ...   ...          ...         ...        ...   
49995           90   155    31   230    194 190 9  155 31 230          B   
49996          202   147    92   191    186 58 89  147 92 191          A   
49997          184   243    14   105    241 39 21  243 14 105          A   
49998           96   204    22    41   204 158 13   204 22 41          B   
49999           95    42   130   186    192 4 211  42 130 186          A   

      d_ip_class s_port_class d_port_class cluster_label_kmeans  \
0              B   registered   registered                    1   
1              A   registered   registered                    1   
2              A      dynamic   well-known                    2   
3              A   registered   registered                    1   
4              A      dynamic   well-known                   18   
...          ...          ...          ...                  ...   
49995          A   registered   registered                    1   
49996          C   registered   registered                    1   
49997          B   registered      dynamic                    1   
49998          A   registered   registered                    1   
49999          A   registered   registered                    1   

      has_sql_injection has_shell_shock has_malicious_execution  \
0                     0               0                       0   
1                     0               0                       0   
2                     1               0                       0   
3                     0               0                       0   
4                     0               1                       0   
...                 ...             ...                     ...   
49995                 0               0                       0   
49996                 0               0                       0   
49997                 0               0                       0   
49998                 0               0                       0   
49999                 0               0                       0   

      has_xss_attack has_xml_rpc has_directory_traversal has_shell_file  
0                  0           0                       0              0  
1                  0           0                       0              0  
2                  0           0                       0              0  
3                  0           0                       0              0  
4                  0           0                       0              0  
...              ...         ...                     ...            ...  
49995              0           0                       0              0  
49996              0           0                       0      

In [ ]:
y

class
0      4_unknown
1      4_unknown
2      2_exploit
3      4_unknown
4      2_exploit
...          ...
49995  4_unknown
49996  4_unknown
49997  4_unknown
49998  4_unknown
49999  4_unknown

[50000 rows x 1 columns]

In [ ]:
x_train = x.copy()
y_train = y.copy()

In [ ]:
print(x_train.shape)
print(y_train.shape)

(50000, 26)
(50000, 1)


In [ ]:
from sklearn.model_selection import train_test_split, KFold

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
cv = KFold(n_splits=5, shuffle=True, random_state=999)

### 오버샘플링

In [ ]:
'''
from imblearn.over_sampling import SMOTEN
sampler = SMOTEN(random_state=42, n_jobs=-1)
x_res, y_res = sampler.fit_resample(x_train, y_train)
y_res.value_counts()
'''

In [ ]:
'''
overxy = pd.concat([x_res, y_res], axis=1)
overxy.to_csv('overxy.csv')

x_train = x_res
y_train = y_res
'''

In [ ]:
cat_features = [ col for col in x_train.columns if x_train[col].dtypes == "object"]
cat_features

['s_port',
 'd_port',
 'protocol',
 'risk',
 's_network_id',
 's_ip2',
 's_ip3',
 's_ip4',
 'd_network_id',
 'd_ip2',
 'd_ip3',
 'd_ip4',
 's_host_ip',
 'd_host_ip',
 's_ip_class',
 'd_ip_class',
 's_port_class',
 'd_port_class',
 'cluster_label_kmeans',
 'has_sql_injection',
 'has_shell_shock',
 'has_malicious_execution',
 'has_xss_attack',
 'has_xml_rpc',
 'has_directory_traversal',
 'has_shell_file']

In [ ]:
scores = []

models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=2000,random_state=999,eval_metric="TotalF1", cat_features=cat_features)
    model.fit(x_train.iloc[tri], y_train.iloc[tri], 
            eval_set=[(x_train.iloc[vai], y_train.iloc[vai])], 
            early_stopping_rounds=100 ,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["TotalF1"])
    if False:
        break

Learning rate set to 0.089202
0:	learn: 0.9804691	test: 0.9797990	best: 0.9797990 (0)	total: 604ms	remaining: 20m 7s
100:	learn: 0.9914781	test: 0.9912160	best: 0.9914233 (53)	total: 46.5s	remaining: 14m 33s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9914232816
bestIteration = 53

Shrink model to first 54 iterations.
Learning rate set to 0.089202
0:	learn: 0.9798414	test: 0.9805956	best: 0.9805956 (0)	total: 560ms	remaining: 18m 39s
100:	learn: 0.9912141	test: 0.9923934	best: 0.9923934 (100)	total: 50s	remaining: 15m 39s
200:	learn: 0.9919413	test: 0.9923100	best: 0.9924998 (143)	total: 1m 36s	remaining: 14m 25s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9924998348
bestIteration = 143

Shrink model to first 144 iterations.
Learning rate set to 0.089202
0:	learn: 0.9796590	test: 0.9810830	best: 0.9810830 (0)	total: 580ms	remaining: 19m 18s
100:	learn: 0.9914914	test: 0.9928025	best: 0.9928025 (93)	total: 48.5s	remaining: 15m 12s
Stoppe

In [ ]:
import joblib
joblib.dump(model, "cat2.pkl")
#model = joblib.load("cat2.pkl")

['cat2.pkl']

In [ ]:
from sklearn.metrics import classification_report

y_pred_tr= model.predict(x_train)
print(classification_report(y_train, y_pred_tr))

y_pred_te= model.predict(x_test)
print(classification_report(y_test, y_pred_te))

                  precision    recall  f1-score   support

1_reconnaissance       0.98      0.97      0.97      2502
       2_exploit       0.99      1.00      1.00     19691
          3_post       1.00      1.00      1.00     11718
       4_unknown       1.00      1.00      1.00     16089

        accuracy                           1.00     50000
       macro avg       0.99      0.99      0.99     50000
    weighted avg       1.00      1.00      1.00     50000

                  precision    recall  f1-score   support

1_reconnaissance       0.97      0.96      0.97       627
       2_exploit       0.99      1.00      1.00      4841
          3_post       1.00      1.00      1.00      2954
       4_unknown       1.00      1.00      1.00      4078

        accuracy                           1.00     12500
       macro avg       0.99      0.99      0.99     12500
    weighted avg       1.00      1.00      1.00     12500



In [ ]:
y_pred_tr = pd.DataFrame(y_pred_tr)
y_pred_tr.columns = ['predclass']

In [ ]:
crosstab = pd.crosstab(y_pred_tr['predclass'], y['class'], rownames=['real'], colnames=['pred'])
crosstab

pred              1_reconnaissance  2_exploit  3_post  4_unknown
real                                                            
1_reconnaissance              2416         42      12          5
2_exploit                       81      19635      11          7
3_post                           5         14   11677          6
4_unknown                        0          0      18      16071

In [ ]:
from sklearn.metrics import precision_score, f1_score
print(precision_score(y_pred_tr, y_train, average = "macro"))
print(precision_score(y_pred_te, y_test, average = "macro"))
print(f1_score(y_pred_tr, y_train, average = "macro"))
print(f1_score(y_pred_te, y_test, average = "macro"))

0.989541472936648
0.9893899079618897
0.9907493967499746
0.9897765306230548


In [ ]:
pre = model.predict(x2)

In [ ]:
pre = pd.DataFrame(pre)

In [ ]:
pre.value_counts()

2_exploit           3981
3_post              3513
4_unknown           1723
1_reconnaissance     783
dtype: int64

In [ ]:
pre = pre.reset_index()

In [ ]:
pre.columns = ['idx', 'class']

In [ ]:
pre.head()

idx             class
0    0            3_post
1    1            3_post
2    2  1_reconnaissance
3    3         4_unknown
4    4         2_exploit

In [ ]:
pre.to_csv('pre.csv', index=False)

In [ ]:
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(x_test.columns)[sorted_idx])
plt.title('Feature Importance')

Text(0.5, 1.0, 'Feature Importance')

In [ ]:
predt=pd.DataFrame(model.predict(x_test))
predt.columns = ['predclass']
predt.set_index(y_test.index, inplace=True)
predcon = pd.concat([y_test, predt], axis = 1)

In [ ]:
crosstab = pd.crosstab(predcon['class'], predt['predclass'], rownames=['real'], colnames=['pred'])
crosstab

### Isolation Forest(IF)

#### 예선의 테스트셋은 레이블이 주어지지 않기 때문에,  각 레이블별로 IF를 돌릴수 없음.
#### 그래서 아래와 같은 방법을 생각함.
#### 1. 예선전 데이터를 train/test로 나눈다음 train셋을 각 레이블별로 나눔
#### 2. 각 레이블별로 IF를 돌려서 이상치를 제거함
#### 3. IF로 제거한 이상치가 없는 데이터로 모델을 학습해 보고 테스트셋의 정확도를 확인해보려 함.

In [ ]:
# 설명변수 모두 범주형 변수로 변환(수치형으로 보이는 자료 없음)
x.info()

In [ ]:
xiso = x.copy()

In [ ]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
class MultiColLabelEncoder:
    def __init__(self):
        self.encoder_dict = defaultdict(LabelEncoder)

    def fit_transform(self, X: pd.DataFrame, columns: list):
        if not isinstance(columns, list):
            columns = [columns]

        output = X.copy()
        output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].fit_transform(x))

        return output

In [ ]:
def inverse_transform(self, X: pd.DataFrame, columns: list):
    if not isinstance(columns, list):
        columns = [columns]

    if not all(key in self.encoder_dict for key in columns):
        raise KeyError(f'At least one of {columns} is not encoded before')

    output = X.copy()
    try:
        output[columns] = X[columns].apply(lambda x: self.encoder_dict[x.name].inverse_transform(x))       
    except ValueError:
        print(f'Need assignment when do "fit_transform" function') 
        raise

    return output

In [ ]:
catfeature = [col for col in xiso.columns if x_train[col].dtypes == "object"]
catfeature

In [ ]:
mle=MultiColLabelEncoder()
xiso = mle.fit_transform(xiso, columns = catfeature)

In [ ]:
xiso_train, xiso_test, yiso_train, yiso_test = train_test_split(xiso, y, test_size = 0.25, random_state = 42)

In [ ]:
iftrain = pd.concat([xiso_train, yiso_train], axis = 1)
iftrain['class'].value_counts()

iftrain4 = iftrain[iftrain['class']=='4_unknown']
iftrain4.drop('class', axis=1, inplace=True)

iftrain3 = iftrain[iftrain['class']=='3_unknown']
iftrain3.drop('class', axis=1, inplace=True)

iftrain2 = iftrain[iftrain['class']=='2_exploit']
iftrain2.drop('class', axis=1, inplace=True)

iftrain1 = iftrain[iftrain['class']=='1_reconnaissance']
iftrain1.drop('class', axis=1, inplace=True)

iftrain4.reset_index(inplace=True)
iftrain3.reset_index(inplace=True)
iftrain2.reset_index(inplace=True)
iftrain1.reset_index(inplace=True)

from sklearn.ensemble import IsolationForest

ano = IsolationForest(n_estimators=1000, n_jobs=-1, verbose=1)
train_predif = pd.DataFrame(ano.fit_predict(iftrain1))
train_predif = pd.DataFrame(train_predif)
train_predif.columns = ['iso']
iftrain1 = pd.concat([iftrain1, train_predif], axis=1)

train_predif = pd.DataFrame(ano.fit_predict(iftrain2))
train_predif = pd.DataFrame(train_predif)
train_predif.columns = ['iso']
iftrain2 = pd.concat([iftrain2, train_predif], axis=1)

print(iftrain1['iso'].value_counts())
print('----------------------------')
print(iftrain2['iso'].value_counts())

### 틀린것들 제거 후 훈련

In [ ]:
extrain = pd.concat([x_train, y_train], axis = 1)
extrain.reset_index(inplace=True)

In [ ]:
y_pred_tr =  pd.DataFrame(model.predict(x_train))
y_pred_tr.columns = ['pred_class']

In [ ]:
extrain.head()

In [ ]:
extrain2 = pd.concat([extrain, pd.DataFrame(y_pred_tr['pred_class'])], axis=1)
extrain2 = extrain2[extrain2['class'] != y_pred_tr['pred_class']]
extrain2.to_csv('extrain2.csv')

In [ ]:
x_train.drop(extrain2['index'], axis=0, inplace=True)
y_train.drop(extrain2['index'], axis=0, inplace=True)

In [ ]:
scores = []

models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=2000,random_state=999,eval_metric="TotalF1",cat_features=cat_features)
    model.fit(x_train.iloc[tri], y_train.iloc[tri], 
            eval_set=[(x_train.iloc[vai], y_train.iloc[vai])], 
            early_stopping_rounds=50 ,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["TotalF1"])
    if False:
        break

Learning rate set to 0.112743
0:	learn: 0.9788235	test: 0.9786082	best: 0.9786082 (0)	total: 87.4ms	remaining: 2m 54s
100:	learn: 0.9889186	test: 0.9862106	best: 0.9863309 (59)	total: 1.92s	remaining: 36.1s
200:	learn: 0.9905429	test: 0.9873072	best: 0.9873176 (189)	total: 3.64s	remaining: 32.6s



KeyboardInterrupt



In [ ]:
from sklearn.metrics import classification_report

y_pred_tr= model.predict(x_train)
print(classification_report(y_train, y_pred_tr))

y_pred_te= model.predict(x_test)
print(classification_report(y_test, y_pred_te))

In [ ]:
from sklearn.metrics import precision_score, f1_score
print(precision_score(y_pred_tr, y_train, average = "macro"))
print(precision_score(y_pred_te, y_test, average = "macro"))
print(f1_score(y_pred_tr, y_train, average = "macro"))
print(f1_score(y_pred_te, y_test, average = "macro"))

#### 잘못 예측한 값 추출

In [ ]:
'''
x2 = pd.read_csv("track_b_learn.csv").drop(["idx"],axis=1)
x3 = pd.read_csv('train_touch.csv')
y2 = pd.read_csv("track_b_learn_label.csv")
y_train["pred_class"]=y_pred_tr
x3.reset_index(inplace=True)
x2.reset_index(inplace=True)
'''

In [ ]:
'''
false_train = y_train[y_train["class"] != y_train["pred_class"]]
false_train.reset_index(inplace=True)
'''

In [ ]:
x3

In [ ]:
'''
merge_train = pd.merge(x2, false_train, on='index')
merge_train = pd.merge(x3[['index', 'cluster_label_kmeans']], merge_train, on='index')
merge_train.to_csv('merge_train.csv')
'''

In [ ]:
'''
y_test["pred_class"] = y_pred_te
false_test = y_test[y_test["class"] != y_test["pred_class"]]
false_test.reset_index(inplace=True)
merge_test = pd.merge(x2, false_test, on='index')
merge_test = pd.merge(x3[['index', 'cluster_label_kmeans']], merge_test, on='index')
merge_test.to_csv('merge_test.csv')
'''

#### 파라미터 최적화

In [ ]:
'''
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK
from hyperopt import hp
from hyperopt import fmin, tpe, Trials

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 999)
cv = KFold(n_splits=5, shuffle=True, random_state=999)
'''

In [ ]:
'''
# lgbmClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환

def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    lgbm_clf =  LGBMClassifier(n_estimators=100, num_leaves=int(search_space['num_leaves']),
                               max_depth=int(search_space['max_depth']),
                               min_child_samples=int(search_space['min_child_samples']), 
                               subsample=search_space['subsample'],
                               learning_rate=search_space['learning_rate'])
    accuracy = cross_val_score(lgbm_clf, X_train, y_train, scoring='accuracy', cv=3)
    
    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}
'''

In [ ]:
'''
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves', 32, 128, 1),
                     'max_depth': hp.quniform('max_depth', 100, 300, 1),
                     'min_child_samples': hp.quniform('min_child_samples', 60, 200, 1),
                     'subsample': hp.uniform('subsample', 0.7, 2),
                     'learning_rate': hp.uniform('learning_rate', 0.01, 0.3)
                    }
'''

In [ ]:
'''
lgbm_clf =  LGBMClassifier(n_estimators=500, num_leaves=int(best['num_leaves']),
                           max_depth=int(best['max_depth']),
                           min_child_samples=int(best['min_child_samples']), 
                           subsample=round(best['subsample'], 5),
                           learning_rate=round(best['learning_rate'], 5)
                          )

# evaluation metric을 auc로, early stopping은 100 으로 설정하고 학습 수행. 
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=500, 
            eval_metric="auc_mu",eval_set=[(X_train, y_train), (X_test, y_test)])
'''

#### 페이로드 중복값 확인

In [ ]:
'''
dfd = pd.read_csv('track_b_learn.csv')
dfd.head()
dudu = dfd[dfd.duplicated('payload', keep=False)]
dudu.sort_values(by=['payload'], inplace=True)
dudu.to_csv('dudu.csv')
'''

#### 범주형 변수 상관관계 분석

In [ ]:
x = pd.read_csv('train_touch.csv')
df = pd.concat([x,y],axis = 1)
df

In [ ]:
from scipy.stats import chi2_contingency

def cramers_V(var1,var2) :
    crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None)) # Cross table building
    stat = chi2_contingency(crosstab)[0] # Keeping of the test statistic of the Chi2 test
    obs = np.sum(crosstab) # Number of observations
    mini = min(crosstab.shape)-1 # Take the minimum value between the columns and the rows of the cross table
    return (stat/(obs*mini))

In [ ]:
import numpy as np

data_encoded = ['s_ip', 's_port', 'd_ip', 'd_port', 'protocol', 'payload', 'risk',
       's_network_id', 's_ip2', 's_ip3', 's_ip4', 'd_network_id', 'd_ip2',
       'd_ip3', 'd_ip4', 's_host_ip', 'd_host_ip', 's_ip_class', 'd_ip_class',
       's_port_class', 'd_port_class', 'cluster_label_kmeans', 'has_sql_injection',
       'has_shell_shock', 'has_malicious_execution', 'has_xss_attack',
       'has_xml_rpc', 'has_directory_traversal',
       'has_shell_file', 's_country_code', 'd_country_code']

rows= []

for var1 in data_encoded:
    col = []
    cramers =cramers_V(df[var1], df["class"]) # Cramer's V test
    col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V  
    rows.append(col)
    
cramers_results = np.array(rows)

corr = pd.DataFrame(cramers_results, columns = ["class"], index = data_encoded )

corr

In [ ]:
corr = corr.sort_values(by = "class", ascending = False)

corr[corr.values == 0].index
corr0 = ['d_country_code', 's_country_code', 'has_shell_file',
       'has_directory_traversal', 's_network_id', 'has_web_shell', 's_ip2',
       'd_network_id', 's_ip_class', 'd_ip_class']

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(6,12))
with sns.axes_style("white"):
    ax = sns.heatmap(corr, mask=mask,vmin=0., vmax=1, square=True)

plt.show()

### API를 이용한 IP별 국가 피쳐 생성

In [ ]:
df1 = pd.read_csv('track_b_pre.csv')
df1.head()

In [ ]:
df1.shape

In [ ]:
import requests
import time

# https://app.abstractapi.com/api/ip-geolocation/documentation
api_url = "https://ipgeolocation.abstractapi.com/v1"
api_keys = ["975e11d693004de7b3188e9344ef41b4", "92a77d54633b4036916593da62b5a9d3"]
len_api_keys = len(api_keys)

def find_country_code(api_key, ip):
    time.sleep(1)

    response = requests.get(f"{api_url}/?api_key={api_key}&ip_address={ip}").json()
    print(response)
    return response.get("country_code")

In [ ]:
import pathlib
output_csv_path = "ip_location_pre2.csv"
output = pd.read_csv(output_csv_path) if pathlib.Path(output_csv_path).exists() else None

from numpy import nan
import ipaddress
from multiprocessing import Pool

s_ip = df1["s_ip"]
d_ip = df1["d_ip"]
len_df1 = len(df1)

columns = ["idx" , "s_country_code", "d_country_code"]
df = pd.DataFrame(list(), columns= columns) if output is None else output
len_df = len(df)

try:
    for i in range(5943, 7501): # len_df1
        s_country_code = None if len_df <= i else df["s_country_code"][i]
        d_country_code = None if len_df <= i else df["d_country_code"][i]
        if (s_country_code is not None and not pd.isna(s_country_code)) and (d_country_code is not None and not pd.isna(d_country_code)):
            continue

        print(f"\n{i}")
        s_country_code = find_country_code(api_keys[0], s_ip[i])
        d_country_code = find_country_code(api_keys[1], d_ip[i]) 
        df.loc[i] = [i, s_country_code, d_country_code]    
except KeyboardInterrupt:
    pass

df.to_csv(output_csv_path, sep=',', index=False)

# https://www.freecodecamp.org/news/how-to-get-location-information-of-ip-address-using-python/